In [29]:
# %pip install bertopic
# %pip install indicnlp-library
# %pip install transformers

In [30]:
# get processed data

import os
import pandas as pd

file_path = "../data/"

data = pd.read_csv(file_path+"train.csv")

data

,news_id,news_date,news_category,news_title,news_article
0,6,1/6/2011 2:45:49 PM,மர்மம்,தூக்கில் தொங்கும் சேவல்கள் திருடர்களை காவு வாங...,"நாலு ஆள் உயரம், முறுக்கு மீசை, கையில் வீச்சரிவ..."
1,9,1/6/2011 2:56:51 PM,மர்மம்,பவுர்ணமி ஜாமத்தில் மாயமான கர்ப்பிணி,அமானுஷ்யமான சம்பவங்கள் நம்மை சுற்றி ஆங்காங்கே ...
2,11,1/6/2011 3:08:15 PM,மர்மம்,மச்சுபிச்சு மலை ரகசியம்,தென்அமெரிக்க நாடான பெருவில் காடுகள் மிகவும் பய...
3,12,1/6/2011 3:09:20 PM,மர்மம்,ரத்த பலி வாங்கும் விபரீத ஆவி,கடந்த 18ம் தேதி சாயங்காலம்... அடைமழையை கிழித்த...
4,13,1/6/2011 3:11:00 PM,மர்மம்,உலகப் பேரழகியின் மர்ம மரணம்,வரலாற்று பேரழகிகள் பட்டியலில் இன்றும் முதலிடத்...
...,...,...,...,...,...
49995,59792,7/16/2014 4:22:14 PM,உலகம்,லட்சம் கோடி முதலீட்டில் பிரிக்ஸ் வளர்ச்சி வங்க...,போர்டலிசா: பிரிக்ஸ் கூட்டமைப்பு நாடுகளின் வளர்...
49996,59793,7/16/2014 4:22:48 PM,தமிழகம்,ஆயிரம் விலைபஸ் ஸ்டாண்டில் கூவி கூவி குழந்தையை ...,திருமலை:பஸ் நிலையத்தில் பச்சிளம் பெண் குழந்தைக...
49997,59795,7/16/2014 4:23:57 PM,தமிழகம்,கோடி பறிமுதல் செய்யப்பட்ட வழக்கு கரகாட்டக்காரி...,வேலூர்:வேலூரை சேர்ந்த கரகாட்டக்காரி மோகனம்மாள்...
49998,59796,7/16/2014 4:59:42 PM,தமிழகம்,எழும்பூர் ரயில் நிலையத்தில் பிரீபெய்டு ஆட்டோ,சென்னை:சென்னை எழும்பூர் ரயில்நிலையத்தில் பிரீப...


In [31]:
# Convert to list
df=data["news_title"]

docs = [str(i) for i in df.values]

In [32]:

import torch
from transformers import BertModel, BertTokenizerFast

tokenizer = BertTokenizerFast.from_pretrained("setu4993/LaBSE")
model = BertModel.from_pretrained("setu4993/LaBSE")

# Set model to evaluation mode
model.eval()


"""
Encode a batch of documents using LaBSE model and return their embeddings.
Args:
    docs: List of strings representing the documents to be encoded.
    batch_size: Size of the batch to be used during encoding.
Returns:
    embeddings: Tensor of shape (n_docs, embedding_size) representing the document embeddings.
"""
# Encode the documents in batches
n_docs = len(docs)
batch_size = 8
embeds = torch.zeros((n_docs, model.config.hidden_size))
for i in range(0, n_docs, batch_size):
    batch = docs[i:i+batch_size]
    inputs = tokenizer(batch, return_tensors="pt", padding=True)
    with torch.no_grad():
        outputs = model(**inputs)
    batch_embeddings = outputs.pooler_output
    embeds[i:i+batch_size] = batch_embeddings





In [33]:

file_path = "../data/"

# save embeddings

import pickle

with open(file_path+'embeddings.pkl', 'wb') as f:

    pickle.dump(embeds, f)

    



In [34]:
# Fine tune UMAP - DIMENSIONALITY REDUCTION STEP

from umap import UMAP


umap_model = UMAP(n_neighbors=3, n_components=3, min_dist=0.05)



In [35]:
# Fine tune HDBSCAN - CLUSTERING STEP

from hdbscan import HDBSCAN

hdbscan_model = HDBSCAN(min_cluster_size=60, min_samples=30,
                        prediction_data=True, gen_min_span_tree=True)

In [36]:
# Tokenize docs trivially (split on spaces)

from indicnlp.tokenize import sentence_tokenize, indic_tokenize

def tokenize_ta(text,return_tensors="pt",*args,**kwargs):
    return indic_tokenize.trivial_tokenize(text)

In [37]:
# Common list of stopwords

stopwords=['அங்கு',
 'அங்கே',
 'அடுத்த',
 'அதனால்',
 'அதன்',
 'அதற்கு',
 'அதிக',
 'அதில்',
 'அது',
 'அதே',
 'அதை',
 'அந்த',
 'அந்தக்',
 'அந்தப்',
 'அன்று',
 'அல்லது',
 'அவன்',
 'அவரது',
 'அவர்',
 'அவர்கள்',
 'அவள்',
 'அவை',
 'ஆகிய',
 'ஆகியோர்',
 'ஆகும்',
 'இங்கு',
 'இங்கே',
 'இடத்தில்',
 'இடம்',
 'இதனால்',
 'இதனை',
 'இதன்',
 'இதற்கு',
 'இதில்',
 'இது',
 'இதை',
 'இந்த',
 'இந்தக்',
 'இந்தத்',
 'இந்தப்',
 'இன்னும்',
 'இப்போது',
 'இரு',
 'இருக்கும்',
 'இருந்த',
 'இருந்தது',
 'இருந்து',
 'இவர்',
 'இவை',
 'உன்',
 'உள்ள',
 'உள்ளது',
 'உள்ளன',
 'எந்த',
 'என',
 'எனக்',
 'எனக்கு',
 'எனப்படும்',
 'எனவும்',
 'எனவே',
 'எனினும்',
 'எனும்',
 'என்',
 'என்ன',
 'என்னும்',
 'என்பது',
 'என்பதை',
 'என்ற',
 'என்று',
 'என்றும்',
 'எல்லாம்',
 'ஏன்',
 'ஒரு',
 'ஒரே',
 'ஓர்',
 'கொண்ட',
 'கொண்டு',
 'கொள்ள',
 'சற்று',
 'சிறு',
 'சில',
 'சேர்ந்த',
 'தனது',
 'தன்',
 'தவிர',
 'தான்',
 'நான்',
 'நாம்',
 'நீ',
 'பற்றி',
 'பற்றிய',
 'பல',
 'பலரும்',
 'பல்வேறு',
 'பின்',
 'பின்னர்',
 'பிற',
 'பிறகு',
 'பெரும்',
 'பேர்',
 'போது',
 'போன்ற',
 'போல',
 'போல்',
 'மட்டுமே',
 'மட்டும்',
 'மற்ற',
 'மற்றும்',
 'மிக',
 'மிகவும்',
 'மீது',
 'முதல்',
 'முறை',
 'மேலும்',
 'மேல்',
 'யார்',
 'வந்த',
 'வந்து',
 'வரும்',
 'வரை',
 'வரையில்',
 'விட',
 'விட்டு',
 'வேண்டும்',
 'வேறு']

from sklearn.feature_extraction.text import CountVectorizer

# Create a vectorizer object to generate term document counts for topic representation - TOKENIZATION STEP

vectorizer_model = CountVectorizer(
    stop_words=stopwords,analyzer='word',
    tokenizer=tokenize_ta
)

In [38]:
import pandas as pd
from scipy.sparse import csr_matrix
from typing import List, Mapping, Tuple, Union
import stanza

class TamilPOS():
    """
    Extract Topic Keywords based on their Part-of-Speech using stanza library for Tamil.
    """
    def __init__(self,
                 top_n_words: int = 10,
                 pos_patterns: List[str] = None):
        self.top_n_words = top_n_words

        if pos_patterns is None:
            self.pos_patterns = [
                'NOUN',
                'PROPN',
                'ADJ',
            ]
        else:
            self.pos_patterns = pos_patterns

        # load stanza pipeline for Tamil

        self.nlp = stanza.Pipeline(lang='ta', processors='tokenize,pos')

    def extract_topics(self,
                       topic_model,
                       documents: pd.DataFrame,
                       c_tf_idf: csr_matrix,
                       topics: Mapping[str, List[Tuple[str, float]]]
                       ) -> Mapping[str, List[Tuple[str, float]]]:
        topic_to_keywords = {}
        for topic_id, topic_words in topics.items():

            # filter candidate documents that contain at least one keyword from the topic

            mask = documents['text'].str.contains('|'.join([word[0] for word in topic_words]), regex=True)
            candidate_docs = documents[mask]

            # extract candidate keywords from candidate_docs based on POS patterns

            candidate_keywords = []
            for doc in candidate_docs['text']:
                doc_keywords = []

                # get POS tags for each word in the document

                doc_words = self.nlp(doc).sentences[0].words
                for word in doc_words:
                    if word.upos in self.pos_patterns:
                        doc_keywords.append(word.text)
                candidate_keywords.extend(doc_keywords)

            # count the frequency of each keyword and keep the top n

            candidate_keyword_counts = pd.Series(candidate_keywords).value_counts().head(self.top_n_words)

            # normalize keyword counts

            candidate_keyword_counts = candidate_keyword_counts / candidate_docs.shape[0]

            # assign c-TF-IDF scores to keywords

            keyword_scores = [(word, topic_model.get_topic(topic_id)[word]) for word in candidate_keyword_counts.index]

            # sort keywords by their respective c-TF-IDF scores

            sorted_keyword_scores = sorted(keyword_scores, key=lambda x: x[1], reverse=True)

            # add top n keywords to topic_to_keywords dict
            
            topic_to_keywords[topic_id] = sorted_keyword_scores[:self.top_n_words]
        return topic_to_keywords


In [39]:
representation_model = TamilPOS()

2023-04-30 21:18:11 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES
2023-04-30 21:18:11 WARNING: Language ta package default expects mwt, which has been added
2023-04-30 21:18:12 INFO: Loading these models for language: ta (Tamil):
| Processor | Package |
-----------------------
| tokenize  | ttb     |
| mwt       | ttb     |
| pos       | ttb     |

2023-04-30 21:18:12 INFO: Using device: cpu
2023-04-30 21:18:12 INFO: Loading: tokenize
2023-04-30 21:18:12 INFO: Loading: mwt
2023-04-30 21:18:12 INFO: Loading: pos
2023-04-30 21:18:12 INFO: Done loading processors!


In [40]:

# Create a BERTopic model

from bertopic import BERTopic

topic_model = BERTopic(
    verbose=True,
    calculate_probabilities=False,
    embedding_model=model,
    vectorizer_model=vectorizer_model,
    umap_model=umap_model,
    hdbscan_model=hdbscan_model,
)

In [41]:
# Fit the model on the documents

embeds_np = embeds.detach().numpy()
topics = topic_model.fit_transform(docs,embeds_np)

2023-04-30 21:18:20,273 - BERTopic - Reduced dimensionality


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

2023-04-30 21:18:22,237 - BERTopic - Clustered reduced embeddings


In [42]:
# Get the topics

topic_model.get_topic_info()

,Topic,Count,Name
0,-1,23676,-1_கைது_கொலை_பெண்_அரசு
1,0,912,0_ஆஸி_சதம்_வாட்சன்_டிராவிட்
2,1,780,1_ஒபாமா_பாகிஸ்தான்_அமெரிக்க_பாகிஸ்தானில்
3,2,707,2_மின்_மக்கள்_குடிநீர்_மறியல்
4,3,703,3_மோதி_லாரி_பலி_பைக்
...,...,...,...
138,137,63,137_மகன்_தந்தை_கொன்ற_கைது
139,138,62,138_பெட்ரோல்_வீச்சு_குண்டு_நிர்வாகி
140,139,61,139_மாவட்டத்தில்_திருவள்ளூர்_லிட்டர்_தண்ணீர்
141,140,61,140_நோயாளிகள்_மருத்துவமனையில்_ஆஸ்பத்திரியில்_அரசு


In [43]:
representation_model=TamilPOS(pos_patterns=['NOUN'])

2023-04-30 21:18:24 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES
2023-04-30 21:18:24 WARNING: Language ta package default expects mwt, which has been added
2023-04-30 21:18:25 INFO: Loading these models for language: ta (Tamil):
| Processor | Package |
-----------------------
| tokenize  | ttb     |
| mwt       | ttb     |
| pos       | ttb     |

2023-04-30 21:18:25 INFO: Using device: cpu
2023-04-30 21:18:25 INFO: Loading: tokenize
2023-04-30 21:18:25 INFO: Loading: mwt
2023-04-30 21:18:25 INFO: Loading: pos
2023-04-30 21:18:25 INFO: Done loading processors!


In [44]:
topic_model.update_topics(docs,representation_model=representation_model,vectorizer_model=vectorizer_model)

In [45]:
topic_model.get_topic_info()

,Topic,Count,Name
0,-1,23676,-1_கைது_கொலை_பெண்_அரசு
1,0,912,0_ஆஸி_சதம்_வாட்சன்_டிராவிட்
2,1,780,1_ஒபாமா_பாகிஸ்தான்_அமெரிக்க_பாகிஸ்தானில்
3,2,707,2_மின்_மக்கள்_குடிநீர்_மறியல்
4,3,703,3_மோதி_லாரி_பலி_பைக்
...,...,...,...
138,137,63,137_மகன்_தந்தை_கொன்ற_கைது
139,138,62,138_பெட்ரோல்_வீச்சு_குண்டு_நிர்வாகி
140,139,61,139_மாவட்டத்தில்_திருவள்ளூர்_லிட்டர்_தண்ணீர்
141,140,61,140_நோயாளிகள்_மருத்துவமனையில்_ஆஸ்பத்திரியில்_அரசு


In [46]:
import dill 

with open("../data/model.pkl", "wb") as f:
    model = dill.dump(topic_model, f)